In [1]:
import pymongo
import pandas
from pprint import pprint

# Connect
client = pymongo.MongoClient(host="mongo", port=27017, username="imdb", password="imdb_admin")

In [2]:
df1 = pandas.read_csv("IMDB-movies.csv")
df2 = pandas.read_csv("IMDB-directors.csv")
df3 = pandas.read_csv("IMDB-movies_directors.csv")
df4 = pandas.read_csv("IMDB-movies_genres.csv")
df5 = df4.merge(df1).merge(df3).merge(df2)

df4 = df4.merge(df1)
df3 = df3.merge(df1).merge(df2)

db = client['imdb_database']
collection1 = db['imdb_movies']
collection2 = db['imdb_directors']
collection3 = db['imdb_movies_directors']
collection4 = db['imdb_movies_genres']
collection5 = db['imdb_movies_directors_genres']


df1.reset_index(inplace=True)
data_dict = df1.to_dict("records")# Insert collection
collection1.insert_many(data_dict)

df2.reset_index(inplace=True)
data_dict = df2.to_dict("records")# Insert collection
collection2.insert_many(data_dict)

df3.reset_index(inplace=True)
data_dict = df3.to_dict("records")# Insert collection
collection3.insert_many(data_dict)

df4.reset_index(inplace=True)
data_dict = df4.to_dict("records")# Insert collection
collection4.insert_many(data_dict)

df5.reset_index(inplace=True)
data_dict = df5.to_dict("records")# Insert collection
collection5.insert_many(data_dict)

# Introduction to MongoDB (MongoDB Query Language)

##### Version 0.1

***

By Scott Coughlin (Northwestern IT Research Computing Services)  
20 July 2022

In our introduction to MongoDB we will start with queries of existing tables.

## Problem 1) IMDb Data

Throughout the session we will use information from the [Internet Movie Database (IMDb)](https://www.imdb.com/) to illustrate various principles regarding databases.

A quick note on the provenance of this data. The files we have used to populate this data set are from [this website](https://relational.fit.cvut.cz/dataset/IMDb) and it may not be a list of every single movie on IMDb (there are no movies after 2004).

For this exercise there are 5 collections, 
```
collection1 = db['imdb_movies']
collection2 = db['imdb_directors']
collection3 = db['imdb_movies_directors']
collection4 = db['imdb_movies_genres']
collection5 = db['imdb_movies_directors_genres']
```
To make things simple, I have already performed the necessary steps to "join" the information from imdb_movies and imdb_directories together to make a bigger collection "imdb_movies_directors" and so on

#### HELPFUL TIP: Convery all resulting queries to a pandas.DataFrame by wrapping the `pymongo` query in

```
df = pandas.DataFrame(list(db.imdb_movies_genres.find())
print(df)
	_id 	index 	director_id 	first_name 	last_name
0 	62da05f8e5d6d03453887957 	70115 	71645 	Martin 	Scorsese
```

### Second Helpful Tip: See the MongoDB SQL to Mongo mapping information to help: https://www.mongodb.com/docs/manual/reference/sql-comparison/

In [3]:
pandas.DataFrame(list(db.imdb_directors.find({"last_name" : "Scorsese"})))

,_id,index,director_id,first_name,last_name
0,62dabcc36195866c4cc7e4ad,70115,71645,Martin,Scorsese


**Problem 1a**

Using pymongo, SELECT 10 movies from the imbd_movies table. Select 10 directors from imbd_directors and order by `first_name`.

In [4]:
pandas.DataFrame(list(db.imdb_movies.find().limit(10)))

,_id,index,movie_id,name,year,rank
0,62dabcb66195866c4cc16780,0,0,#28,2002,0.0
1,62dabcb66195866c4cc16781,1,1,"#7 Train: An Immigrant Journey, The",2000,0.0
2,62dabcb66195866c4cc16782,2,2,$,1971,6.4
3,62dabcb66195866c4cc16783,3,3,"$1,000 Reward",1913,0.0
4,62dabcb66195866c4cc16784,4,4,"$1,000 Reward",1915,0.0
5,62dabcb66195866c4cc16785,5,5,"$1,000 Reward",1923,0.0
6,62dabcb66195866c4cc16786,6,6,"$1,000,000 Duck",1971,5.0
7,62dabcb66195866c4cc16787,7,7,"$1,000,000 Reward, The",1920,0.0
8,62dabcb66195866c4cc16788,8,8,"$10,000 Under a Pillow",1921,0.0
9,62dabcb66195866c4cc16789,9,9,"$100,000.00",1915,0.0


In [5]:
pandas.DataFrame(list(db.imdb_directors.find().sort('first_name').limit(10)))

,_id,index,director_id,first_name,last_name
0,62dabcc16195866c4cc6e2c1,4087,4175,A.,Balakrishnan
1,62dabcc16195866c4cc6ecca,6656,6779,A.,Berry
2,62dabcc16195866c4cc6f18a,7872,8026,A.,Bobrov
3,62dabcc16195866c4cc6ee22,7000,7125,A.,Bhimsingh
4,62dabcc16195866c4cc6e572,4776,4871,A.,Barr-Smith
5,62dabcc16195866c4cc6ef89,7359,7494,A.,Bistritsky
6,62dabcc16195866c4cc70664,13210,13475,A.,Chandrasekaran
7,62dabcc16195866c4cc705ed,13091,13355,A.,Champeaux
8,62dabcc16195866c4cc6e108,3646,3728,A.,Babes
9,62dabcc16195866c4cc6d70c,1090,1114,A.,Aleksandrov


**Problem 1b**

Using pymongo, how many movies are there? How many directors are there? 

In [6]:
db.imdb_movies.find().count()

<ipython-input-6-f218dfc85700>:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  db.imdb_movies.find().count()


355146

In [7]:
db.imdb_directors.find().count()

<ipython-input-7-3b483e236fde>:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  db.imdb_directors.find().count()


86880

*Write your answer here*

**Problem 1c**

Using pymongo, determine how many movies are there after the year 2000?

In [8]:
db.imdb_movies.count( { 'year': { '$gt': 2000 } } )

<ipython-input-8-9c22225c26c1>:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  db.imdb_movies.count( { 'year': { '$gt': 2000 } } )


39586

*Write your answer here*

**Problem 1d**

How many different movie genres are there?

In [9]:
db.imdb_movies_genres.distinct( "genre" )

['Action',
 'Adult',
 'Adventure',
 'Animation',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Short',
 'Thriller',
 'War',
 'Western']

*Write your answer here*

*write your answer here*

## Problem 2) Groups and Aggregates

Now that we know why the data has been organized in this way, we can leverage this unique structure in order to learn interesting properties of the data. 

**Problem 3a**

In which year were the most movies made according to IMDb?

In [10]:
pandas.DataFrame(db.imdb_movies.aggregate([
{'$group': {
      '_id': { 'year' : "$year" },
        'count': { '$count': { } },
    }},
{'$sort' : { 'count': -1 }
 }
]))

,_id,count
0,{'year': 2002},10337
1,{'year': 2003},10119
2,{'year': 2000},10107
3,{'year': 2001},10002
4,{'year': 1999},9389
...,...,...
115,{'year': 1891},6
116,{'year': 1890},3
117,{'year': 1893},2
118,{'year': 1888},2


*write your answer here*

**Problem 3b**

How many "Action" movies where made after the year 1980? Before the year 1980?

In [11]:
pandas.DataFrame(db.imdb_movies_genres.aggregate([
{'$match' : { 'year': {'$gt' : 1980} }},
{'$group': {
      '_id': { 'genre' : "$genre" },
        'count': { '$count': { } },
    }},
]))

,_id,count
0,{'genre': 'Mystery'},2122
1,{'genre': 'Thriller'},7503
2,{'genre': 'Comedy'},21169
3,{'genre': 'Drama'},32463
4,{'genre': 'Horror'},4700
5,{'genre': 'Crime'},5881
6,{'genre': 'Romance'},6208
7,{'genre': 'Adventure'},4030
8,{'genre': 'Sci-Fi'},3420
9,{'genre': 'Adult'},6437


In [12]:
pandas.DataFrame(db.imdb_movies_genres.aggregate([
{'$match' : { 'year': {'$lt' : 1980} }},
{'$group': {
      '_id': { 'genre' : "$genre" },
        'count': { '$count': { } },
    }},
]))

,_id,count
0,{'genre': 'Mystery'},2696
1,{'genre': 'Thriller'},2906
2,{'genre': 'Comedy'},34782
3,{'genre': 'Film-Noir'},410
4,{'genre': 'Drama'},39520
5,{'genre': 'Crime'},6490
6,{'genre': 'Horror'},2239
7,{'genre': 'Romance'},7301
8,{'genre': 'Adult'},835
9,{'genre': 'Sci-Fi'},1405


*write your answer here*

**Problem 3c**

Select all films made by `Scorsese`. How many are there?

In [13]:
db.imdb_movies_directors.find({"first_name" : "Martin", "last_name" : "Scorsese"}).count()

<ipython-input-13-a4d6fe10e67a>:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  db.imdb_movies_directors.find({"first_name" : "Martin", "last_name" : "Scorsese"}).count()


38

*write your answer here*

**Problem 3c**

According the the IMDb data, which director has directed the most movies?

In [14]:
pandas.DataFrame(db.imdb_movies_directors.aggregate([
{'$group': {
      '_id': { 'director_id' : "$director_id" , 'last_name': "$last_name"},
        'count': { '$count': { } },
    }},
    {'$sort' : { 'count': -1 }},
]))


,_id,count
0,"{'director_id': 25116, 'last_name': 'Fleischer'}",615
1,"{'director_id': 56530, 'last_name': 'Méliès'}",554
2,"{'director_id': 30570, 'last_name': 'Griffith'}",530
3,"{'director_id': 1958, 'last_name': 'Anderson'}",360
4,"{'director_id': 24576, 'last_name': 'Feuillade'}",345
...,...,...
77429,"{'director_id': 6447, 'last_name': 'Berg-Hilli...",1
77430,"{'director_id': 85285, 'last_name': 'White'}",1
77431,"{'director_id': 9571, 'last_name': 'Brennan'}",1
77432,"{'director_id': 51227, 'last_name': 'Masseroni'}",1


*write your answer here*

**Problem 3d**

According the the IMDb data, which director has directed the most movies in each genre?

In [15]:
pandas.DataFrame(db.imdb_movies_directors_genres.aggregate([
{'$group': {
      '_id': { 'last_name': "$last_name", 'genre': "$genre", 'director_id' : "$director_id" , },
        'count': { '$count': { } },
    }},
        {'$sort' : { 'count': -1 }},
]))


,_id,count
0,"{'last_name': 'Fleischer', 'genre': 'Animation...",611
1,"{'last_name': 'Fleischer', 'genre': 'Short', '...",604
2,"{'last_name': 'Méliès', 'genre': 'Short', 'dir...",454
3,"{'last_name': 'Griffith', 'genre': 'Short', 'd...",448
4,"{'last_name': 'Kneitel', 'genre': 'Animation',...",315
...,...,...
154773,"{'last_name': 'Rioyo', 'genre': 'War', 'direct...",1
154774,"{'last_name': 'Segar', 'genre': 'Western', 'di...",1
154775,"{'last_name': 'Dubinkin', 'genre': 'Drama', 'd...",1
154776,"{'last_name': 'Heeley', 'genre': 'Music', 'dir...",1


*write your answer here*

## Challenge Problem) Make your own tables

**Problem 1a**

Create a new "collection".

In [23]:
new_collection = db["new_collection"]

**Problem 1b**

INSERT 3 "documentions" into the "collection you made above

In [24]:
new_collection.insert_one({"user_id" :  "abc123"})
new_collection.insert_one({"user_id" :  "axyz123"})
new_collection.insert_one({"user_id" :  "456123"})

**Problem 1c**

Create a pandas DataFrame and save the rows as "documents" in a new "collection" you make

*** hint look at using `pandas.to_dict` ****